In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import normalize
import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, Input, Dropout
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier
from keras.activations import relu, sigmoid

In [2]:
df = pd.read_csv(r'C:\Users\marku\Desktop\ML\MLGit\datasets\heart.csv')

In [3]:
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


# EDA in Oblig2 HeartDataset folder

In [4]:
features = df.drop('output',axis=1)
targets = df['output']

targets_onehot = pd.get_dummies(targets)
features_onehot = pd.get_dummies(features)
targets_onehot = targets_onehot.astype('float32')
features_onehot = features_onehot.astype('float32')

features_onehot.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
0,63.0,1.0,3.0,145.0,233.0,1.0,0.0,150.0,0.0,2.3,0.0,0.0,1.0
1,37.0,1.0,2.0,130.0,250.0,0.0,1.0,187.0,0.0,3.5,0.0,0.0,2.0
2,41.0,0.0,1.0,130.0,204.0,0.0,0.0,172.0,0.0,1.4,2.0,0.0,2.0
3,56.0,1.0,1.0,120.0,236.0,0.0,1.0,178.0,0.0,0.8,2.0,0.0,2.0
4,57.0,0.0,0.0,120.0,354.0,0.0,1.0,163.0,1.0,0.6,2.0,0.0,2.0


In [5]:
features_onehot = normalize(features_onehot)

train_features, test_features, train_targets, test_targets = train_test_split(features_onehot, targets_onehot, test_size=0.25, random_state=42)

In [6]:
train_features_tensor = tf.convert_to_tensor(train_features)
test_features_tensor = tf.convert_to_tensor(test_features)
train_targets_tensor = tf.convert_to_tensor(train_targets)
test_targets_tensor = tf.convert_to_tensor(test_targets)

# NN Architecture 1

In [11]:
model1 = Sequential()

model1.add(Input(13))

model1.add(Dense(26, activation='relu'))
model1.add(Dense(26, activation='relu'))
model1.add(Dense(26, activation='relu'))
model1.add(Dense(13, activation='relu'))

model1.add(Dense(2, activation='sigmoid'))

In [1175]:
opt1 = Adam()
model1.compile(optimizer=opt1, loss='binary_crossentropy', metrics=['accuracy'])

In [1176]:
model1.fit(train_features_tensor, train_targets_tensor, epochs=500, batch_size=6)

Epoch 1/500
38/38 [==============================] - 1s 2ms/step - loss: 0.6946 - accuracy: 0.4670
Epoch 2/500
38/38 [==============================] - 0s 2ms/step - loss: 0.6916 - accuracy: 0.6035
Epoch 3/500
38/38 [==============================] - 0s 1ms/step - loss: 0.6905 - accuracy: 0.5991
Epoch 4/500
38/38 [==============================] - 0s 1ms/step - loss: 0.6896 - accuracy: 0.5947
Epoch 5/500
38/38 [==============================] - 0s 2ms/step - loss: 0.6872 - accuracy: 0.5859
Epoch 6/500
38/38 [==============================] - 0s 2ms/step - loss: 0.6844 - accuracy: 0.6256
Epoch 7/500
38/38 [==============================] - 0s 2ms/step - loss: 0.6812 - accuracy: 0.6300
Epoch 8/500
38/38 [==============================] - 0s 2ms/step - loss: 0.6774 - accuracy: 0.6564
Epoch 9/500
38/38 [==============================] - 0s 1ms/step - loss: 0.6705 - accuracy: 0.6388
Epoch 10/500
38/38 [==============================] - 0s 1ms/step - loss: 0.6621 - accuracy: 0.6564
Epoch 11/

In [ ]:
## Atempt at Automatic Hyperparameter tuning

In [1179]:
model1.evaluate(train_features_tensor, train_targets_tensor)

8/8 [==============================] - 0s 930us/step - loss: 0.3044 - accuracy: 0.8590


[0.30444276332855225, 0.8590308427810669]

In [1180]:
model1.evaluate(test_features_tensor, test_targets_tensor)

3/3 [==============================] - 0s 2ms/step - loss: 0.4081 - accuracy: 0.8553


[0.408107727766037, 0.8552631735801697]

Somehow the test accuracy was higher then the training. I fixed this by upping the testing data size from 0.1 to 0.25 and downsizing the batch size to 6.

## Atempt at Automatic Hyperparameter tuning

In [10]:
def create_model1(learning_rate, layer1_neurons, layer2_neurons, layer3_neurons, layer4_neurons):
    model = Sequential([
        Input(13),
        Dense(layer1_neurons, activation='relu'),
        Dense(layer2_neurons, activation='relu'),
        Dense(layer3_neurons, activation='relu'),
        Dense(layer4_neurons, activation='relu'),
        Dense(2, activation='sigmoid')
    ])

    opt1 = Adam(learning_rate=learning_rate)
    model.compile(optimizer=opt1, loss='binary_crossentropy', metrics=['accuracy'])

    return model

classifier = KerasClassifier(create_model1, epochs=200, batch_size=10)
params = {
    'learning_rate':[0.01,0.1,0.2,0.3],
    'layer1_neurons':[15,30,60],
    'layer2_neurons':[15,30,60],
    'layer3_neurons':[15,30,60],
    'layer4_neurons':[15,30,60]
}

grid = GridSearchCV(classifier, params)
grid.fit(train_features_tensor, train_targets_tensor)

TypeError: __init__() got an unexpected keyword argument 'my_params'

# NN Architecture 2

In [1332]:
model2 = Sequential()

model2.add(Input(13))

model2.add(Dense(26, activation='relu'))
model2.add(Dropout(0.3))

model2.add(Dense(13, activation='relu'))
model2.add(Dense(2, activation='sigmoid'))

In [1333]:
opt1 = Adam()
model2.compile(optimizer=opt1, loss='binary_crossentropy', metrics=['accuracy'])

In [1337]:
model2.fit(train_features_tensor, train_targets_tensor, epochs=500, batch_size=15)

Epoch 1/500
16/16 [==============================] - 0s 915us/step - loss: 0.3928 - accuracy: 0.8326
Epoch 2/500
16/16 [==============================] - 0s 824us/step - loss: 0.3963 - accuracy: 0.8150
Epoch 3/500
16/16 [==============================] - 0s 803us/step - loss: 0.4143 - accuracy: 0.8018
Epoch 4/500
16/16 [==============================] - 0s 1ms/step - loss: 0.4183 - accuracy: 0.8150
Epoch 5/500
16/16 [==============================] - 0s 2ms/step - loss: 0.3888 - accuracy: 0.8150
Epoch 6/500
16/16 [==============================] - 0s 2ms/step - loss: 0.3822 - accuracy: 0.8106
Epoch 7/500
16/16 [==============================] - 0s 1ms/step - loss: 0.3901 - accuracy: 0.8458
Epoch 8/500
16/16 [==============================] - 0s 2ms/step - loss: 0.3948 - accuracy: 0.8150
Epoch 9/500
16/16 [==============================] - 0s 1ms/step - loss: 0.3977 - accuracy: 0.8414
Epoch 10/500
16/16 [==============================] - 0s 1ms/step - loss: 0.3861 - accuracy: 0.8238
Epo

In [1338]:
model2.evaluate(train_features_tensor, train_targets_tensor)

8/8 [==============================] - 0s 787us/step - loss: 0.3150 - accuracy: 0.8590


[0.3150014281272888, 0.8590308427810669]

In [1339]:
model2.evaluate(test_features_tensor, test_targets_tensor)

3/3 [==============================] - 0s 999us/step - loss: 0.3882 - accuracy: 0.8553


[0.38821926712989807, 0.8552631735801697]

This model performed the exact same as the first architecture, but with less layers.

## Atempt at Automatic Hyperparameter tuning

In [15]:
def create_model2(learning_rate, layer1_neurons, layer2_neurons, activation_function, drop_out):
    model = Sequential([
        Input(15),
        Dense(layer1_neurons, activation=activation_function),
        Dropout(drop_out),
        Dense(layer2_neurons, activation=activation_function),
        Dense(2, activation='sigmoid')
    ])

    opt1 = Adam(learning_rate=learning_rate)
    model.compile(optimizer=opt1, loss='binary_crossentropy', metrics=['accuracy'])

    return model

classifier = KerasClassifier(create_model2, epochs=200, batch_size=10)
params = {
    'learning_rate':[0.01,0.1,0.2,0.3],
    'drop_out':[0.2,0.3,0.4],
    'layer1_neurons':[15,30,60],
    'layer2_neurons':[15,30,60],
    'activation_function':[relu, sigmoid]
}
grid = GridSearchCV(classifier, params)
grid.fit(train_features_tensor, train_targets_tensor)

C:\Users\marku\AppData\Local\Temp\ipykernel_26340\1265838042.py:15: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(create_model2, epochs=200, batch_size=10)


TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got array([ 46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,
        59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,
        72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,
        85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
        98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110,
       111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123,
       124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136,
       137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149,
       150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162,
       163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175,
       176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188,
       189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201